## Load LLM

In [ ]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

class CustomOllama(Ollama):
    def __init__(self, model, callback_manager=None, stop=None, temperature=0.2):
        super().__init__(model=model, stop=stop, temperature=temperature)
        self.callback_manager = callback_manager

# Initialize the CallbackManager
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Create an instance of the CustomOllama class
llm = CustomOllama(
    model="llama3",
    callback_manager=callback_manager,
    stop=[""]
)

temperature = 0.2
#higher temperature = more creativity 

In [ ]:
#test 
prompt = ["Why is the sky blue?"]  # Prompt should be a list of strings

# Generate text using the Ollama model
generated_text = llm.generate(prompts=prompt, temperature=temperature)

## Setup RAG

In [ ]:
plain_retriever = vectorstore.as_retriever()

### Prompt Template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You're a helpful AI assistant for a jobseeker with a criminal conviction trying to understand what legal restrictions exist for jobs and certifications."
    "Given a user question and sections of Virginia law code, answer the user question. If none of the sections of the law code from Virginia answer the question"
    "Just say you don't know."
    "\n\nUse only the following sections of the law code to answer the question. Provide the source from the context: "
    "{context}"
    )

prompt_plain = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
prompt_plain.pretty_print()

### Chain

In [ ]:
from typing import List, Dict
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableMap

# Function to format documents with their metadata
def format_docs(docs: List[Document]):
    formatted_docs = []
    for doc in docs:
        metadata_str = "\n".join(f"{key}: {value}" for key, value in doc.metadata.items())
        formatted_doc = f"Metadata:\n{metadata_str}\n\nContent:\n{doc.page_content}"
        formatted_docs.append(formatted_doc)
    return "\n\n".join(formatted_docs)

# Chain to format documents, process with LLM, and parse the output
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt_plain  # Assuming 'prompt' is defined somewhere in your setup
    | llm  # Assuming 'llm' is defined somewhere in your setup
    | StrOutputParser()
)

# Function to retrieve documents based on a query
retrieve_docs = (lambda x: x["input"]) | retriever  # Assuming 'retriever' is defined somewhere in your setup

# Define the final chain
chain_plain = RunnablePassthrough.assign(context=retrieve_docs).assign(
    answer=rag_chain_from_docs
)

### Test with prompt

In [ ]:
# Assuming you have a string for your prompt template
query = "What job restrictions exist for someone convicted of a crime?"

In [ ]:
result = chain_plain.invoke({"input": query})

In [ ]:
result['context']

# Trulens Eval

In [ ]:
# Local IP address command 
# !curl ifconfig.me.

In [ ]:
import trulens_eval
# Imports main tools:
from trulens_eval import TruChain, Tru
tru = Tru()
tru.reset_database()

In [ ]:
# Initialize LiteLLM-based feedback function collection class:
from langchain.llms import Ollama
from trulens_eval import LiteLLM
import litellm

In [ ]:
#'http://localhost:11435' refers to everyone's local device
ollama_provider = LiteLLM(model_engine='ollama/llama3', api_base='http://174.20.175.133:11435')

In [ ]:
import numpy as np
from trulens_eval import Feedback, Select

context = result['context']
context_texts = [doc.page_content for doc in context]

# Define a groundedness feedback function
f_groundedness = (
    Feedback(ollama_provider.groundedness_measure_with_cot_reasons, name="Groundedness")
    .on(Select.RecordCalls.args.context)  # This selects the context from the function call
    .on_output()
)

# Question/answer relevance between overall question and answer
f_answer_relevance = (
    Feedback(ollama_provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on(Select.RecordCalls.args.query)  # This selects the query/input
    .on_output()  # This selects the output directly
)

# Context relevance between question and each context chunk
f_context_relevance = (
    Feedback(ollama_provider.context_relevance_with_cot_reasons, name="Context Relevance")
    .on(Select.RecordCalls.args.query)  # This selects the query/input
    .on(Select.RecordCalls.args.context)  # This selects the context
    .aggregate(np.mean)
) 

In [ ]:
tru_recorder = TruChain(
    app=llm,
    app_id='Chain1_ChatApplication',
    feedbacks=[f_answer_relevance, f_context_relevance, f_groundedness], 
    selectors_check_warning=True
)

In [ ]:
with tru_recorder as recording:
    llm_response = chain_plain.invoke({"input": query})

display(llm_response)